# Imports and Authorization

In [0]:
from google.colab import auth, drive
from google.auth import default
import gspread
from googleapiclient.discovery import build

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
service = build('sheets', 'v4', credentials=creds)
drive.mount('/content/drive')

import requests
from bs4 import BeautifulSoup

import time
from datetime import datetime
import pytz

import os
from openpyxl import load_workbook
!pip install pyxirr
from pyxirr import xirr

# Fetch Screener Data ~ 60 mins

In [0]:
class Screener():
  def __init__(self):
    self.base_url = 'https://www.screener.in'
    self.industry_to_url = {}
    self.stock_to_data = {}
    self.epoch_time = time.time()
    self.stock = None

  def get_industry_to_url(self):
    soup = BeautifulSoup(requests.get(self.base_url+'/market').text,'html.parser')
    for ele in soup.find_all('a', href=lambda href: href and href.startswith('/market/')):
      self.industry_to_url[ele.text] = self.base_url + ele['href']

  def get_price_and_ratios(self):
    for ele in self.stock_to_data[self.stock]['Soup'].find_all('span',{'class':'name'}):
      if ele.text.strip() == 'Current Price':
        self.stock_to_data[self.stock]['LTP'] = float(ele.find_next('span',{'class':'number'}).text.replace(',',''))
      elif ele.text.strip() == 'High / Low':
        self.stock_to_data[self.stock]['52W H'] = float(ele.find_all_next('span',{'class':'number'})[0].text.replace(',',''))
        self.stock_to_data[self.stock]['52W L'] = float(ele.find_all_next('span',{'class':'number'})[1].text.replace(',',''))
      elif ele.text.strip() == 'Stock P/E':
        self.stock_to_data[self.stock]['PE'] = float(ele.find_next('span',{'class':'number'}).text.replace(',','')) if ele.find_next('span',{'class':'number'}).text.replace(',','')!='' else 0

  def get_quarterly_results(self):
    section = self.stock_to_data[self.stock]['Soup'].find('section',{'id':'quarters'})
    self.stock_to_data[self.stock]['Reported_Upto'] = section.find('table',{'class':'data-table responsive-text-nowrap'}).find('thead').find_all('th')[-1].text.strip()
    if section.find('span',{'class':'badge'}):
      self.stock_to_data[self.stock]['Upcoming_Date'] = section.find('span',{'class':'badge'}).find('strong').text

  def get_profit_loss(self):
    section = self.stock_to_data[self.stock]['Soup'].find('section',{'id':'profit-loss'})
    for row in section.find('table',{'class':'data-table responsive-text-nowrap'}).find_all('tr'):
      if not self.stock_to_data[self.stock]['Reported_Upto']:
        self.stock_to_data[self.stock]['Reported_Upto'] = row.find_all('th')[-1].text.strip()
      button = row.find('button')
      if button:
        txt = button['onclick'].strip()
        type = txt[txt.find("('")+2:txt.find("',")]
      else:
        type = row.find('td').text.strip() if row.find('td') else None
      data = [float(col.text.strip().replace(',','').replace('%',''))
        for col in row.find_all('td')[1:] if col.text.strip()]
      if type in ['Sales','Revenue'] and len(data)>1:
        self.stock_to_data[self.stock]['Sales_TTM'] = data[-1]
      elif type in ['Net Profit'] and len(data)>1:
        self.stock_to_data[self.stock]['Profit_TTM'] = data[-1]
    for tbl in section.find_all('table',{'class':'ranges-table'}):
      th = tbl.find('th').text
      rows = tbl.find_all('td')
      if th=='Compounded Sales Growth':
        self.stock_to_data[self.stock]['Sales_Growth_10Y'] = rows[1].text.strip().replace('%','')
        self.stock_to_data[self.stock]['Sales_Growth_5Y'] = rows[3].text.strip().replace('%','')
        self.stock_to_data[self.stock]['Sales_Growth_3Y'] = rows[5].text.strip().replace('%','')
        self.stock_to_data[self.stock]['Sales_Growth_TTM'] = rows[7].text.strip().replace('%','')
      elif th=='Compounded Profit Growth':
        self.stock_to_data[self.stock]['Profit_Growth_10Y'] = rows[1].text.strip().replace('%','')
        self.stock_to_data[self.stock]['Profit_Growth_5Y'] = rows[3].text.strip().replace('%','')
        self.stock_to_data[self.stock]['Profit_Growth_3Y'] = rows[5].text.strip().replace('%','')
        self.stock_to_data[self.stock]['Profit_Growth_TTM'] = rows[7].text.strip().replace('%','')

  def get_margin_data(self):
    if self.stock_to_data[self.stock]['Sales_TTM'] not in [0,None]:
      self.stock_to_data[self.stock]['NPM_TTM'] = round((self.stock_to_data[self.stock]['Profit_TTM']/self.stock_to_data[self.stock]['Sales_TTM']),2)*100

  def get_fii_data(self):
    tbl = self.stock_to_data[self.stock]['Soup'].find('section',{'id':'shareholding'}).find('table',{'class':'data-table'})
    if tbl:
      for row in tbl.find_all('tr'):
        button = row.find('button')
        if button:
          txt = button['onclick'].strip()
          type = txt[txt.find("('")+2:txt.find("',")]
        else:
          type = row.find('td').text.strip() if row.find('td') else None
        data = [float(col.text.strip().replace(',','').replace('%',''))
          for col in row.find_all('td')[1:] if col.text.strip()]
        if type == 'foreign_institutions' and len(data)>7:
          fii_ttm = round(sum(data[-4:])/4,2)
          fii_pttm = round(sum(data[-8:-4])/4,2)
          fii_ttm_pttm = fii_ttm-fii_pttm
          self.stock_to_data[self.stock]['FII_TTM_PTTM'] = fii_ttm_pttm

  def get_stock_to_data(self):
    for i, (industry, url) in enumerate(self.industry_to_url.items(), start=1):
      time.sleep(1)
      stocks_url = {ele.text : self.base_url+ele['href']
        for ele in BeautifulSoup(requests.get(url).text,'html.parser').find_all('a', href=lambda href: href and href.startswith('/company/'))}
      for s, (self.stock, stock_url) in enumerate(stocks_url.items(), start=1):
        time.sleep(0.1)
        soup = BeautifulSoup(requests.get(stock_url).text,'html.parser')
        self.stock_to_data[self.stock] = {'url':stock_url,'Industry':industry,'Soup':soup, 'symbol':stock_url.split('/')[4],
                                          'LTP':None,'52W L':None,'52W H':None,'PE':None,
                                          'Reported_Upto':None,'Upcoming_Date':None,
                                          'Sales_TTM':None,'Profit_TTM':None,'NPM_TTM':None,
                                          'Sales_Growth_10Y':None,'Sales_Growth_5Y':None,'Sales_Growth_3Y':None,'Sales_Growth_TTM':None,
                                          'Profit_Growth_10Y':None,'Profit_Growth_5Y':None,'Profit_Growth_3Y':None,'Profit_Growth_TTM':None,
                                          'FII_TTM_PTTM':None}
        print(f"\r{industry}[{i}/{len(self.industry_to_url)}] {self.stock}[{s}/{len(stocks_url)}] ", end="", flush=True)
        self.get_price_and_ratios()
        self.get_quarterly_results()
        self.get_profit_loss()
        self.get_margin_data()
        self.get_fii_data()
        self.stock_to_data[self.stock]['Soup'] = None
        if not self.stock_to_data[self.stock]['LTP']:
          print(stock_url)

scraper = Screener()
scraper.get_industry_to_url()
scraper.get_stock_to_data()

# Update Screener Data

In [0]:
doc = gc.open('Screener Tracker')
stocks = doc.worksheet('Stocks')
stocks.clear()
stocks.clear_notes(['A','H','S'])
stocks.clear_basic_filter()
stocks_cells = []
stocks_notes = {}
stocks_cells.append(gspread.Cell(row=1,col=1,value='Stock'))
timestamp = datetime.fromtimestamp(scraper.epoch_time,tz=pytz.timezone('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
stocks_notes['A1'] = f'Timestamp : {timestamp}'
i = ((datetime.today().month-1)//3-1)%4
last_quarter = ['Mar ','Jun ','Sep ','Dec '][i]+str(datetime.today().year-(i==3))
stocks_cells.append(gspread.Cell(row=1,col=2,value=f'Last\nTraded\nPrice'))
stocks_cells.append(gspread.Cell(row=1,col=3,value=f'52\nWeek\nLow'))
stocks_cells.append(gspread.Cell(row=1,col=4,value=f'52\nWeek\nHigh'))
stocks_cells.append(gspread.Cell(row=1,col=5,value=f'Normal\nScore'))
stocks_notes['E1'] = '(LTP-52WL) X 100\n--------------------------\n(52WH-52WL)'
stocks_cells.append(gspread.Cell(row=1,col=6,value='P/E'))
stocks_cells.append(gspread.Cell(row=1,col=7,value=f'Sales\nTTM\n(Cr)'))
stocks_cells.append(gspread.Cell(row=1,col=8,value=f'Profit\nTTM\n(Cr)'))
stocks_cells.append(gspread.Cell(row=1,col=9,value=f'NPM\nTTM\n%'))
stocks_cells.append(gspread.Cell(row=1,col=10,value=f'Sales\nCAGR\n10Y%'))
stocks_cells.append(gspread.Cell(row=1,col=11,value=f'Sales\nCAGR\n5Y%'))
stocks_cells.append(gspread.Cell(row=1,col=12,value=f'Sales\nCAGR\n3Y%'))
stocks_cells.append(gspread.Cell(row=1,col=13,value=f'Sales\nCAGR\nTTM%'))
stocks_cells.append(gspread.Cell(row=1,col=14,value=f'Profit\nCAGR\n10Y%'))
stocks_cells.append(gspread.Cell(row=1,col=15,value=f'Profit\nCAGR\n5Y%'))
stocks_cells.append(gspread.Cell(row=1,col=16,value=f'Profit\nCAGR\n3Y%'))
stocks_cells.append(gspread.Cell(row=1,col=17,value=f'Profit\nCAGR\nTTM%'))
stocks_cells.append(gspread.Cell(row=1,col=18,value=f'FIIs\nChange\nTTM'))
stocks_cells.append(gspread.Cell(row=1,col=19,value=f'Reported Upto/\nUpcoming Date'))
stocks_cells.append(gspread.Cell(row=1,col=20,value=f'Industry'))

row_num = 1
for stock, data in scraper.stock_to_data.items():
  row_num += 1
  stocks_cells.append(gspread.Cell(row=row_num,col=1,value='=HYPERLINK("' + str(data['url']) + '","' + str(data['symbol']) + '")'))
  stocks_notes[f'A{row_num}'] = stock
  stocks_cells.append(gspread.Cell(row=row_num,col=2,value=str(data['LTP'])))
  stocks_cells.append(gspread.Cell(row=row_num,col=3,value=int(data['52W L'])))
  stocks_cells.append(gspread.Cell(row=row_num,col=4,value=int(data['52W H'])))
  stocks_cells.append(gspread.Cell(row=row_num,col=5,value=round((data['LTP']-data['52W L'])/(data['52W H']-data['52W L']) if (data['52W H']-data['52W L'])!=0 else 0,2)*100))
  stocks_cells.append(gspread.Cell(row=row_num,col=6,value=data['PE']))
  stocks_cells.append(gspread.Cell(row=row_num,col=7,value=data['Sales_TTM']))
  stocks_cells.append(gspread.Cell(row=row_num,col=8,value=data['Profit_TTM']))
  stocks_cells.append(gspread.Cell(row=row_num,col=9,value=data['NPM_TTM']))
  stocks_cells.append(gspread.Cell(row=row_num,col=10,value=data['Sales_Growth_10Y']))
  stocks_cells.append(gspread.Cell(row=row_num,col=11,value=data['Sales_Growth_5Y']))
  stocks_cells.append(gspread.Cell(row=row_num,col=12,value=data['Sales_Growth_3Y']))
  stocks_cells.append(gspread.Cell(row=row_num,col=13,value=data['Sales_Growth_TTM']))
  stocks_cells.append(gspread.Cell(row=row_num,col=14,value=data['Profit_Growth_10Y']))
  stocks_cells.append(gspread.Cell(row=row_num,col=15,value=data['Profit_Growth_5Y']))
  stocks_cells.append(gspread.Cell(row=row_num,col=16,value=data['Profit_Growth_3Y']))
  stocks_cells.append(gspread.Cell(row=row_num,col=17,value=data['Profit_Growth_TTM']))
  stocks_cells.append(gspread.Cell(row=row_num,col=18,value=data['FII_TTM_PTTM']))
  stocks_cells.append(gspread.Cell(row=row_num,col=19,value=data['Reported_Upto']))
  if data['Upcoming_Date']:
    stocks_cells.append(gspread.Cell(row=row_num,col=19,value=data['Upcoming_Date']))
  stocks_cells.append(gspread.Cell(row=row_num,col=20,value=data['Industry']))

stocks.update_cells(stocks_cells,value_input_option='USER_ENTERED')
stocks.update_notes(stocks_notes)

# Update Zerodha Data

In [0]:
dividend_statement = [['Symbol','Date','Net Amount','Year-Month']]
account = None
for file in os.listdir('/content/drive/MyDrive/Finance/Zerodha/Dividend Statement'):
  if account is None:
    account = file.split('-')[1]
  workbook = load_workbook(os.path.join('/content/drive/MyDrive/Finance/Zerodha/Dividend Statement', file))
  worksheet = workbook['Equity Dividends']
  found_header = False
  for row in worksheet.iter_rows(values_only=True):
    if not found_header:
      if 'Symbol' in row:
        found_header = True
      continue
    if row[1]=='Total Dividend Amount':
      break
    dividend_statement.append(
      [row[1].replace('#','').replace('6',''),
      row[3], row[6], row[3][:7]])
dividend_statement[1:] = sorted(dividend_statement[1:], key=lambda row: datetime.strptime(row[1], "%Y-%m-%d"), reverse=True)

contract_note = [['Symbol','Date','Net Amount','Year-Month']]
for file in os.listdir('/content/drive/MyDrive/Finance/Zerodha/Contract Note'):
  workbook = load_workbook(os.path.join('/content/drive/MyDrive/Finance/Zerodha/Contract Note', file))
  for worksheet in workbook.worksheets:
    header_found = False
    skip_next_row = False
    sheet_date = datetime.strptime(worksheet.title, "%d-%m-%Y").strftime("%Y-%m-%d")
    for row in worksheet.iter_rows(values_only=True):
      if not header_found:
        if row[0] == 'Order No.':
          header_found = True
          skip_next_row = True
        continue
      if skip_next_row:
        skip_next_row = False
        continue
      if not row[0] or row[0] == 'PAY IN / PAY OUT OBLIGATION':
        break
      contract_note.append([row[4].split(' - ')[0], sheet_date,
        row[11] if row[5]=='buy' else row[12], sheet_date[:7]])
contract_note[1:] = sorted(contract_note[1:], key=lambda row: datetime.strptime(row[1], "%Y-%m-%d"), reverse=True)

holdings = [['Symbol','Quantity','Invested','Current']]
with open('/content/drive/My Drive/Finance/Zerodha/holdings.csv', 'r') as file:
  lines = file.readlines()
for line in lines[1:]:
  parts = line.strip().split(',')
  holdings.append([parts[0].replace('"',''),parts[1],parts[4],parts[5]])

capital_gains = [['Symbol','Date','Invested','Unrealised','Realised','XIRR']]
symbol_to_contracts = {}
symbol_to_holdings = {}
for contract in contract_note[1:]:
  symbol = contract[0]
  if symbol not in symbol_to_contracts:
    symbol_to_contracts[symbol] = []
  symbol_to_contracts[symbol].append(contract)
for holding in holdings[1:]:
  symbol_to_holdings[holding[0]] = holding
for symbol,contracts in symbol_to_contracts.items():
  holdings_invested = 0
  holdings_current = 0
  contracts_invested_held = 0
  contracts_invested_sold = 0
  contracts_sold = 0
  dates = []
  flows = []
  if symbol in symbol_to_holdings:
    holdings_invested = float(symbol_to_holdings[symbol][2])
    holdings_current = float(symbol_to_holdings[symbol][3])
  for c in contracts:
    if c[2]<0:
      if holdings_invested>contracts_invested_held:
        contracts_invested_held += c[2]
        unrealised = (abs(c[2])/holdings_invested)*holdings_current
        xirr_per = round(xirr([datetime.strptime(c[1], "%Y-%m-%d").date(),datetime.today().date()],[c[2],unrealised])*100,2)
        capital_gains.append([symbol,c[1],c[2],unrealised+c[2],0,xirr_per])
      else:
        dates.append(datetime.strptime(c[1], "%Y-%m-%d").date())
        flows.append(c[2])
        contracts_invested_sold += c[2]
    else:
      dates.append(datetime.strptime(c[1], "%Y-%m-%d").date())
      flows.append(c[2])
      contracts_sold += c[2]
  if contracts_sold>0:
    realised = contracts_sold+contracts_invested_sold
    xirr_per = round(xirr(dates, flows)*100,2)
    capital_gains.append([symbol,None,contracts_invested_sold,0,realised,xirr_per])
capital_gains.append(['LTCG',datetime.now().replace(year=datetime.now().year-1).strftime("%Y-%m-%d"),None,None,None,None])
capital_gains[1:] = sorted(capital_gains[1:], key=lambda row: (row[1] is not None, datetime.strptime(row[1], "%Y-%m-%d") if row[1] else None), reverse=True)

doc = gc.open('Personal Finance')
sheet_to_data = {
    'Dividend Statement': dividend_statement,
    'Contract Note': contract_note,
    'Holdings': holdings,
    'Capital Gains': capital_gains
}
for sheet, data in sheet_to_data.items():
  worksheet = doc.worksheet(sheet)
  worksheet.clear()
  worksheet.update(data,'A1')

doc = gc.open('Screener Tracker')
stocks = doc.worksheet('Stocks')
fvs = service.spreadsheets().get(spreadsheetId=doc.id,fields="sheets(filterViews(filterViewId,title))").execute()
for s in fvs.get("sheets", []):
  for fv in s.get("filterViews", []):
    if fv.get("title") in ['Growth Stocks',account]:
      service.spreadsheets().batchUpdate(
          spreadsheetId=doc.id,
          body={"requests": [{"deleteFilterView": {"filterId": fv["filterViewId"]}}]}
      ).execute()
resp = service.spreadsheets().batchUpdate(
    spreadsheetId=doc.id,
    body={"requests": [
    {"addFilterView":
     {"filter":
      { "title": "Growth Stocks",
        "range": {
            "sheetId": stocks.id,
            "startRowIndex": 0,
            "endRowIndex": stocks.row_count,
            "startColumnIndex": 0,
            "endColumnIndex": stocks.col_count
        },
        "criteria": {
            "6": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=G2>1000"}]}},
            "7": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=H2>1000"}]}},
            "8": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=I2>10"}]}},
            "9": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=OR(ISBLANK(J2), J2>10)"}]}},
            "10": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=OR(ISBLANK(K2), K2>10)"}]}},
            "11": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=OR(ISBLANK(L2), L2>10)"}]}},
            "12": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=M2>10"}]}},
            "13": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=OR(ISBLANK(N2), N2>10)"}]}},
            "14": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=OR(ISBLANK(O2), O2>10)"}]}},
            "15": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=OR(ISBLANK(P2), P2>10)"}]}},
            "16": { "condition": {
                    "type": "CUSTOM_FORMULA",
                    "values": [{"userEnteredValue": "=Q2>10"}]
    }}}}}},
    {"addFilterView": {
        "filter": {
            "title": account,
            "range": {
                "sheetId": stocks.id,
                "startRowIndex": 0,
                "endRowIndex": stocks.row_count,
                "startColumnIndex": 0,
                "endColumnIndex": stocks.col_count
            },
            "criteria": {
                "0": {
                    "hiddenValues": list({r[0] for r in service.spreadsheets().values().get(spreadsheetId=doc.id, range='Stocks!A:A').execute().get('values', []) if r}-set(symbol_to_holdings.keys()))
  }}}}}]}).execute()